In [1]:
# pip install geopandas
# import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

In [2]:
token = 'pk.eyJ1IjoianJzdmluY2Vuem8iLCJhIjoiY2wxMHc0ZWMyMDA1MDNqcnN4NDAxZHZxayJ9.XC2yFuBp7YjudgxK2Kjq7Q'

In [3]:
customers = pd.read_csv('CUSTOMERS.csv', sep =';', dtype={"id_ac": str})
customers['count'] = 1

In [4]:
sellers = pd.read_csv('SELLERS.csv', sep = ';',  dtype={'id_ac': str, 'count_sold': 'int32'})
sellers['count'] = 1

In [5]:
# SELLERS DISTRIBUTION

fig = px.scatter_mapbox(sellers, lat = 'lat_seller', lon = 'long_sell',
                         hover_name = 'seller_id', hover_data = ['seller_city'],
                         size='count_sold', color_discrete_sequence=["fuchsia"],
                         zoom=4, opacity = 0.3, size_max = 50,
                         center = {"lat": 40.4167, "lon": -3.7167})
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token,
                  title_text = 'Sellers distribution', title_x = 0.5)
fig.show(renderer='browser')

In [6]:
# CUSTOMER DISTRIBUTION

fig2 = px.scatter_mapbox(customers, lat = 'lat_cust', lon = 'lon_cust',
                         hover_name = 'customer_unique_id',
                         size='order_count', color_discrete_sequence=['turquoise'],
                         zoom=4, opacity = 0.1, size_max = 50,
                         center = {"lat": 40.4167, "lon": -3.7167})
fig2.update_layout(mapbox_style="dark", mapbox_accesstoken=token,
                   title_text = 'Customer distribution', title_x = 0.5)
fig2.show(renderer='browser')

In [16]:
#COUNTING SELLERS PER CITY

sellersxcity = sellers.groupby(['seller_city', 'lat_seller', 'long_sell']).agg({'count_sold':'sum','count':'sum'}).reset_index()

fig3 = px.scatter_mapbox(sellersxcity, lat = 'lat_seller', lon = 'long_sell',
                         hover_name = 'seller_city', hover_data = ['count'],
                         size='count_sold', color_discrete_sequence=["yellow"],
                         zoom=4, opacity = 0.5, size_max = 50,
                         center = {"lat": 40.4167, "lon": -3.7167})

fig3.update_layout(mapbox_style="dark", mapbox_accesstoken=token,
                   title_text = 'Counting sellers per city', title_x = 0.5)
fig3.show(renderer='browser')

In [7]:
#COUNTING ORDERS PER CITY

orderxcity = customers.groupby(['customer_city', 'lat_cust', 'lon_cust','id_ac']).agg({'order_count':'sum','count':'sum'}).reset_index()

fig4 = px.scatter_mapbox(orderxcity, lat = 'lat_cust', lon = 'lon_cust',
                         hover_name = 'customer_city', hover_data = ['count'],
                         size='order_count', color_discrete_sequence=['lime'],
                         zoom=4, opacity = 0.3, size_max = 50,
                         center = {"lat": 40.4167, "lon": -3.7167})

fig4.update_layout(mapbox_style="dark", mapbox_accesstoken=token,
                   title_text = 'Counting orders per city', title_x = 0.5)
fig4.show(renderer='browser')

In [ ]:
# CHOROPLET ORDERS PER AUTONOMOUS COMMUNITY

# gdf = gpd.read_file('map-4.geojson')
# orderxac = customers.groupby(["id_ac"], as_index=False).agg({"order_count": "sum"})

# fig7 = px.choropleth_mapbox(
#    orderxac,
#    geojson=gdf,
#    featureidkey="properties.id_ac",
#    locations="id_ac",
#    color="order_count",
#    color_continuous_scale="matter",
#    zoom=3,
#    center={"lat": 40.4999, "lon": -3.673},
#    labels={"order_count": "Count for Orders"},
# )
# fig7.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# fig7.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
# fig7.show(renderer = 'browser')

In [8]:
# ROUTES OF SELLERS AND VENDORS

orderlist = pd.read_csv('ORDERLIST.csv', sep =';').iloc[:,[0,1,2,5,6,3,7,4,8]]

In [9]:
filtercity = str(input('Provide a city to filter:'))
forderlist = orderlist[orderlist['seller_city'] == filtercity]

In [10]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = forderlist.long_sell, lat = forderlist.lat_sell,
    text = forderlist.seller_city,
    textposition = "bottom right",
    marker = dict(color = 'fuchsia', size = 4)))

fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = forderlist.long_cust, lat = forderlist.lat_cust,
    text = forderlist.customer_city,
    textposition = "bottom right",
    marker = dict(color = 'fuchsia', size = 4)))

i = 0
for i in range(len(forderlist)):
    df = forderlist.iloc[i,:]
    lat = df.iloc[5:7]
    lon = df.iloc[7:9]

    fig.add_trace(go.Scattermapbox(
        mode = "lines",
        lon = lon, lat = lat,
        line=dict(width=2, color = 'fuchsia')))
    i += 1

fig.update_layout(showlegend = False,
                  mapbox=dict(style = 'dark', accesstoken = token,
                              bearing=0, center=go.layout.mapbox.Center(
                                                                    lat=40.4167,
                                                                    lon=-3.7167),
                              pitch=0,
                              zoom=4),
                  title_text = 'Routes of orders from {}'.format(filtercity), title_x = 0.5)
fig.show(renderer='browser')

# orderlist['color'] = 0
# i = 0
# for i in range(len(orderlist)):
#    if orderlist.iloc[i,4] == 'Córdoba':
#        orderlist.iloc[i,-1] = 'red'
#    else:
#        orderlist.iloc[i,-1] = 'green'
#    i =+ 1